In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.init as init
import random

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_knn = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/df_knn.csv')

In [ ]:
# split feature and label
X = df_knn.drop('CVD0010', axis=1)
y = df_knn['CVD0010']

In [ ]:
# normalize X_train, X_test
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
# hyperparameter
n_epochs = 3500
batch_size = 6666
latent_dim = X_train.shape[1]
condition_dim = 1

In [ ]:
# GAN model
# generator_block
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.LeakyReLU(0.2, inplace=True),
    )
# generator
class Generator(nn.Module):
    def __init__(self, z_dim=100, data_dim=latent_dim, c_dim=condition_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim+c_dim, 128),
            get_generator_block(128, 256),
            get_generator_block(256, 512),
            get_generator_block(512, 1024),
            get_generator_block(1024, 2048),
            nn.Linear(2048, data_dim),
        )

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.kaiming_uniform_(m.weight.data, nonlinearity='leaky_relu')
                if m.bias is not None:
                    init.zeros_(m.bias.data)

    def forward(self, noise, condition):
        # Concatenate noise and condition
        input_data = torch.cat([noise, condition], dim=1)
        return self.gen(input_data)

# discriminator_block
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.5)
    )
# 2.discriminator
class Discriminator(nn.Module):
    def __init__(self, data_dim=latent_dim, c_dim=condition_dim):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(data_dim+c_dim, 1024),
            get_discriminator_block(1024, 512),
            get_discriminator_block(512, 256),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    def forward(self, data, condition):
        # Concatenate data and condition
        input_data = torch.cat([data, condition], dim=1)
        return self.disc(input_data)

In [ ]:
# Loss function
criterion = nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Optimizers
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.00001, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.00001, betas=(0.5, 0.999))

# dataloader
my_dataset = TensorDataset(torch.Tensor(X_train.values), torch.Tensor(y_train.values))
dataloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# original gan Train
gen_losses = []
disc_losses = []

for epoch in range(n_epochs):
    for index, (real_data_features, real_data_labels) in enumerate(dataloader):
        real_data_features = real_data_features.float()
        real_data_labels = real_data_labels.float().unsqueeze(-1)

        # Adversarial ground truths
        real_labels = torch.ones(real_data_labels.size(0), 1, requires_grad=False)
        fake_labels = torch.zeros(real_data_labels.size(0), 1, requires_grad=False)

        # -----------------
        #  Train Generator (twice)
        # -----------------

        for _ in range(2):
            g_optimizer.zero_grad()

            # Sample noise as generator input
            z = torch.randn(real_data_features.size(0), 100)
            z_label = torch.tensor(torch.randint(0, 2, (real_data_labels.size(0), 1)))

            # Generate a batch of datas
            gen_datas = generator(z, z_label)

            # Loss measures generator's ability to fool the discriminator
            g_loss = criterion(discriminator(gen_datas, z_label), real_labels)

            g_loss.backward()
            g_optimizer.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        d_optimizer.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = criterion(discriminator(real_data_features, real_data_labels), real_labels)

        # Generate a batch of datas
        z = torch.randn(real_data_features.size(0), 100)
        z_label = torch.tensor(torch.randint(0, 2, (real_data_labels.size(0), 1)))
        gen_datas = generator(z, z_label).detach()

        fake_loss = criterion(discriminator(gen_datas, z_label), fake_labels)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        d_optimizer.step()

        gen_losses.append(g_loss.item())
        disc_losses.append(d_loss.item())

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, index, len(dataloader), d_loss.item(), g_loss.item())
        )


In [ ]:
# Convert lists to NumPy arrays
gen_losses = np.array(gen_losses)
disc_losses = np.array(disc_losses)

# Plot loss curves
plt.figure(figsize=(10, 8))
plt.plot(range(len(gen_losses)), gen_losses, label='Generator loss')
plt.plot(range(len(disc_losses)), disc_losses, label='Discriminator loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
indices = (y_train == 1)
X_real = X_train[indices]
y_real = y_train[indices]

# count the frequency of each value in y_train.
value_counts = y_train.value_counts()
# calculate how much fewer the number of minority classes is than that of majority classes.
diff = value_counts[0] - value_counts[1]

# generate data
z = torch.randn(diff, 100)
z_label = torch.tensor(torch.randint(0, 2, (diff, 1)))
generated_data = generator(z, z_label)
generated_data_df = pd.DataFrame(generated_data.detach().numpy(), columns=X_train.columns)

In [ ]:
# t-SNE
# original data (label==1)
tsne_real = X_real # num == 76

# smote data (label==1)
tsne_real_gan = generated_data_df # num == 392

# TSNE
combined_data = pd.concat([tsne_real_gan, tsne_real], axis=0)
labels = np.array([0] * len(tsne_real_gan) + [1] * len(tsne_real))
tsne = TSNE(n_components=2, random_state=42)
tsne_data = tsne.fit_transform(combined_data)

# show
plt.figure(figsize=(8, 6))
plt.scatter(tsne_data[labels == 0, 0], tsne_data[labels == 0, 1], c='b', label='CGANan data')
plt.scatter(tsne_data[labels == 1, 0], tsne_data[labels == 1, 1], c='r', label='Real data')
plt.legend()
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('TSNE comparing real data with GAN generated data')
plt.show()

In [ ]:
# PCA
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# merge two datas
merged_data = np.concatenate((X_real, generated_data_df), axis=0)

# PCA
pca = PCA(n_components=2)  # dim==2
reduced_data = pca.fit_transform(merged_data)

# show
plt.scatter(reduced_data[X_real.shape[0]:, 0], reduced_data[X_real.shape[0]:, 1], label='CGAN data')
plt.scatter(reduced_data[:X_real.shape[0], 0], reduced_data[:X_real.shape[0], 1], label='Real data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

In [ ]:
value_counts = y_train.value_counts()
diff = value_counts[0] - value_counts[1]

# generate data
z = torch.randn(diff, 100)
z_label = torch.tensor(torch.randint(0, 2, (diff, 1)))
generated_data = generator(z, z_label)
generated_data_df = pd.DataFrame(generated_data.detach().numpy(), columns=X_train.columns)

X_train_cgan = pd.concat([X_train, generated_data_df], axis=0)

ones = pd.Series([1.0] * diff)

y_train_cgan = y_train.append(ones, ignore_index=True)

In [ ]:
from sklearn.utils import shuffle
# shuffle
X_train_cgan, y_train_cgan = shuffle(X_train_cgan, y_train_cgan)

In [ ]:
#SVM
import torch
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score
from sklearn.svm import SVC

def train_and_evaluate_svm(X_train, y_train, X_test, y_test):
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

    # SVM
    svm_classifier = SVC(probability=True)

    # train
    svm_classifier.fit(X_train_tensor.numpy(), y_train_tensor.numpy())

    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    # test
    y_scores = svm_classifier.predict_proba(X_test_tensor.numpy())[:, 1]
    y_pred = svm_classifier.predict(X_test_tensor.numpy())

    # FPR and TPR for ROC curve
    fpr, tpr, thresholds = roc_curve(y_test_tensor.numpy(), y_scores)

    # AUC
    auc = roc_auc_score(y_test_tensor.numpy(), y_scores)

    return auc


In [ ]:
# original data
roc_results = []

# Repeat the training and evaluation process 30 times.
num_repeats = 30
for _ in range(num_repeats):
    auc = train_and_evaluate_svm(X_train, y_train, X_test, y_test)
    roc_results.append(auc)  # Store the AUC value in the tuple

# Calculate the average value of evaluation indicators
mean_auc = np.mean([result for result in roc_results])  # Retrieve the AUC value from the tuple

print("Mean AUC:", mean_auc)

In [ ]:
# gan data
roc_results_cgan = []

# Repeat the training and evaluation process 30 times.
num_repeats = 30
for _ in range(num_repeats):
    auc = train_and_evaluate_svm(X_train_cgan, y_train_cgan, X_test, y_test)
    roc_results_cgan.append(auc)  # Store the AUC value in the tuple

# Calculate the average value of evaluation indicators
mean_auc_cgan = np.mean([result for result in roc_results_cgan])  # Retrieve the AUC value from the tuple

print("CGAN Mean AUC:", mean_auc_cgan)